In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
DEVICE = torch.device("cuda:0")
# Moving to the gpu: net = Net().to(DEVICE)

In [3]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        encode = []
        relu = nn.ReLU(True)
        pool = nn.MaxPool2d(2)
        encode.append(nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(32, 64, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(128, 256, 3, padding=1), relu, pool))
        encode.append(nn.Sequential(nn.Conv2d(256, 256, 3, padding=1), relu))
        
        self.encoders = []
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for n
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for n-1 & n+1
        self.encoders.append(nn.Sequential(*encode).to(DEVICE)) # for D(n-1) & D(n+1)
        
        decode = []
        decode.append(nn.Sequential(nn.Conv2d(256*3, 256, 3, padding = 1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(256*4, 128, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(128*4, 64, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(64*4, 32, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        decode.append(nn.Sequential(nn.ConvTranspose2d(32*4, 1, 3, stride=2, \
                                                       padding=1, output_padding=1), relu))
        self.decoder = nn.Sequential(*decode).to(DEVICE)
        '''
        transform = []
        transform.append(nn.Sequential(nn.Conv3d(1, 32, 3), relu))
        transform.append(nn.Sequential(nn.Conv2d(32, 1, 3), relu))
        
        self.transformer = nn.Sequential(*transform)
        '''
    def forward(self, x):
        '''Fuction uses skips parameter to
        keep track of skip connections between
        encoder and decoder layers.'''
        skips = [[],[],[]]
        nextinput = x
        for i, encoder in enumerate(self.encoders):
            x = nextinput
            for layer in encoder:
                x = layer(x)
                skips[i].append(x)
        x = skips[-1][-1]
        for i, layer in enumerate(self.decoder):
            for j, encoder in enumerate(skips):
                if j == len(skips)-1 and i == 0: break
                x = torch.cat((x,encoder[-(i+1)]),1)
            x = layer(x)
        skips = []
        return x

network = autoencoder()
network = network.to(DEVICE)
x = torch.randn(1,1280,720).to(DEVICE)
network(x.view(-1,1,1280,720))
x.type()

'torch.cuda.FloatTensor'

In [4]:
import os
images = os.listdir('left/skyscraper')
import numpy as np
import matplotlib.pyplot as plt
import cv2

imageL = cv2.imread('left/skyscraper/'+images[50000])
imageR = cv2.imread('right/skyscraper/'+images[50000][0:-4]+'_R.jpg')

plt.imshow(imageL)
plt.show()
plt.imshow(imageR)
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [ ]:
import torch.optim as optim
from tqdm import tqdm

optimizer = optim.Adam(network.parameters(),lr=0.001)# .parameters() takes the parts 
    # that can be optimized, ie
    # they have not been pretrained and frozen.
    # lr or learning rate might decay over time in more complicated situations, .001 is common e-3
    # We wont be decaying it here however!

EPOCHS = 1 # meaning 3 whole passes through the entier dataset
for epoch in range(EPOCHS):
    for frame in tqdm(range(len(images)-100000)):
        imageL = cv2.imread('left/skyscraper/'+images[frame])
        imageR = cv2.imread('right/skyscraper/'+images[frame][0:-4]+'_R.jpg')
        # Data is a featureset paired with images (10 each here)
        Lin = torch.as_tensor(imageL,dtype=float).view(-1,1,1280,720).to(DEVICE)
        Rout = torch.as_tensor(imageR,dtype=float).view(-1,1,1280,720).to(DEVICE)
        # every time befroe passing through, do...
        network.zero_grad()
        # OTHERWISE GRADIENT WILL ONLY GROW
        output = network(Lin.type(torch.cuda.FloatTensor))
        loss = F.smooth_l1_loss(output,Rout.type(torch.cuda.FloatTensor))
        # Two major ways to calculate loss, one is 'one hot vectors' [0 0 0 0 0 1]
        # IFF your dataset is individual numbers use nll, mean sq error for one hot vectors
        loss.backward() # 'Magical' backpropagation done automatically
        optimizer.step() # Adjusts the weights for us!
        if frame % 100 == 0: print(loss)

  0%|          | 2/47312 [00:00<4:31:43,  2.90it/s]

tensor(0.0856, device='cuda:0', grad_fn=<SmoothL1LossBackward>)


  0%|          | 102/47312 [00:39<4:57:06,  2.65it/s]

tensor(48.3196, device='cuda:0', grad_fn=<SmoothL1LossBackward>)


  0%|          | 201/47312 [01:19<6:06:47,  2.14it/s]

...

 49%|████▉     | 23102/47312 [2:33:09<2:33:49,  2.62it/s]

tensor(38.5312, device='cuda:0', grad_fn=<SmoothL1LossBackward>)


 49%|████▉     | 23201/47312 [2:33:48<3:08:11,  2.14it/s]

tensor(39.4988, device='cuda:0', grad_fn=<SmoothL1LossBackward>)


 49%|████▉     | 23233/47312 [2:34:01<2:38:51,  2.53it/s]

In [ ]:
def get_reconstruction(depth_map, imageR):
    '''Uses depth map to reconstruct the left image
    from the right image.'''
    imageL = imageR.copy()
    for i in depth_map.cols:
        for j in depth_map.rows:
            imageL[i][j] = imageR[i+depth_map[i][j]][j]
    return imageL